<a href="https://colab.research.google.com/github/witold87/pub_haystack_unstructurized/blob/develop/unstructured.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install farm-haystack==0.8.0

You should consider upgrading via the '/home/wicio/PycharmProjects/pub_haystack/hay/bin/python -m pip install --upgrade pip' command.


In [5]:
import logging

from haystack.document_store.memory import InMemoryDocumentStore
from haystack.document_store.faiss import FAISSDocumentStore
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
#from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader import TransformersReader
from haystack.reader import FARMReader
from haystack.retriever import DensePassageRetriever
from haystack.retriever import ElasticsearchRetriever
from haystack.utils import print_answers, launch_es
from haystack.preprocessor.cleaning import clean_wiki_text


In [6]:
reader = FARMReader(model_name_or_path="deepset/xlm-roberta-large-squad2", use_gpu=False)

01/27/2022 15:13:36 - INFO - farm.utils -   Using device: CPU 
01/27/2022 15:13:36 - INFO - farm.utils -   Number of GPUs: 0
01/27/2022 15:13:36 - INFO - farm.utils -   Distributed Training: False
01/27/2022 15:13:36 - INFO - farm.utils -   Automatic Mixed Precision: None
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 405, in _handle_workers
    pool._maintain_pool()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 246, in _maintain_pool
    self._repopulate_pool()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 239, in _repopulate_pool
    w.start()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/usr/lib/python3.6/multiprocessing/co

KeyboardInterrupt: 

In [ ]:
import logging

from haystack.document_store.memory import InMemoryDocumentStore
from haystack.document_store.faiss import FAISSDocumentStore
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
#from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader import TransformersReader
from haystack.reader import FARMReader
from haystack.retriever import DensePassageRetriever
from haystack.retriever import ElasticsearchRetriever
from haystack.utils import print_answers, launch_es
from haystack.file_converter.pdf import PDFToTextConverter

#from memory_profiler import profile

def tutorial1_basic_qa_pipeline():
    logger = logging.getLogger(__name__)
    document_store = FAISSDocumentStore(faiss_index_factory_str='Flat')

    from haystack.preprocessor import PreProcessor
    from haystack.file_converter.pdf import PDFToTextConverter

    converter = PDFToTextConverter(remove_numeric_tables=False)
    text = converter.convert(file_path='data/trust_deed_01.pdf', )

    preprocessor = PreProcessor(
        clean_empty_lines=True,
        clean_whitespace=True,
        clean_header_footer=False,
        split_by="word",
        split_overlap=3,
        split_length=100,
        split_respect_sentence_boundary=True
    )

    docs_default = preprocessor.process(text)


    with open('data/nestle_report_de.txt', 'r') as file:
        text = file.readlines()

    text_str = '\n'.join([str(elem) for elem in text])
    stripped_text = text_str.strip('\n')
    print(stripped_text)

    dicts = docs_default
    # dicts = [{'text': stripped_text,
    #           'meta': {'name': 'thai_air_th.txt'}}]
    document_store.write_documents(dicts)
    retriever = DensePassageRetriever(document_store=document_store)
    document_store.update_embeddings(retriever)

    from haystack.pipeline import ExtractiveQAPipeline

    pipe = ExtractiveQAPipeline(reader, retriever)


    # questions = ['Who is in the board of directors?',
    #              'What is the nature of business?',
    #              'When the company was started?',
    #              'Where the company operates?',
    #              'How many workers they have?']

    question_en_trust_deed = [
        'What are the names of the settlors?'
        'What are the names of trustees?'
    ]

    questions_en = ['What is the net debt?',
                    'Wie hoch ist der Umsatz in der Gruppe?',
                    'What is the revenue in group?',
                    'What is the net profit?']


    questions_th = ['หนี้สุทธิคืออะไร??',
                    'รายได้ในกลุ่มคืออะไร?',
                    'กำไรสุทธิคืออะไร?',
                    'how does the covid-19 affect the company?']
    
    questions_de = ['Wie hoch ist die Nettoverschuldung? ',
                    'Wie hoch ist der Umsatz in der Gruppe?' ]


    questions_fr = ['Quelle est la dette nette?',
                    'Quel est le revenu du groupe?',
                    'Quel est le chiffre daffaires du groupe? ']


    questions_es = ['¿Cuáles son el resultado de la crisis?',
                 '¿Cuál es la naturaleza del negocio?',
                 '¿Cuándo se fundó la empresa?',
                 ]

    def predict(questions, pipe):
      for question in questions:
          prediction = pipe.run(
              query=question
          )
          print(prediction['query'])
          print(prediction['answers'][0]['answer'])
          print(prediction['answers'][0]['probability'])
          print(prediction['answers'][0]['context'])

    predict(question_en_trust_deed, pipe)



if __name__ == "__main__":
    tutorial1_basic_qa_pipeline()


In [ ]:
!pip freeze | grep farm

# Nowa sekcja